# Verifica Gerarchie ARX

Questo notebook serve per testare e verificare il caricamento delle gerarchie prima di eseguire l'anonimizzazione completa.


## Importazione Librerie Base

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import glob
import json
from collections import defaultdict

from google.colab import drive
drive.mount('/content/drive')

#base_dir = Path(r"C:\Davide\Università\Magistrale\Secondo Anno\Secondo Semestre\Business Intelligence\Tesina\Progetto Tesina\Schema Matching\generalization\hierarchies")
base_dir = Path("/content/drive/MyDrive/Schema Matching/hierarchies")

print(f"Directory corrente: {Path.cwd()}")
print(f"Directory gerarchie: {base_dir}")

Mounted at /content/drive
Directory corrente: /content
Directory gerarchie: /content/drive/MyDrive/Schema Matching/hierarchies


## Ricerca File delle Gerarchie

In [2]:
# Ricerca tutti i file delle gerarchie
print("File delle gerarchie")

names = ['df1', 'df2']
hierarchy_files = []

for name in names:
    dir_path = base_dir / name
    if not dir_path.exists():
        print(f" Directory non trovata: {dir_path}")
        continue
    files = list(dir_path.glob('*_hierarchy_*.csv'))
    hierarchy_files.extend(files)

print(f"\n Trovati {len(hierarchy_files)} file di gerarchia:")

for file in sorted(hierarchy_files):
    size = file.stat().st_size
    print(f"{file.name} ({size} bytes)")

if not hierarchy_files:
    print("\n Nessun file di gerarchia trovato!")


File delle gerarchie

 Trovati 17 file di gerarchia:
df1_hierarchy_eta_45_normal.csv (2145 bytes)
df1_hierarchy_eta_70_normal.csv (828 bytes)
df1_hierarchy_eta_exponential5.csv (1915 bytes)
df1_hierarchy_eta_uniform5.csv (2237 bytes)
df1_hierarchy_native_exponential.csv (801 bytes)
df1_hierarchy_native_exponential_bias.csv (812 bytes)
df1_hierarchy_native_normal.csv (828 bytes)
df1_hierarchy_native_uniform.csv (896 bytes)
df1_hierarchy_native_uniform_bias.csv (904 bytes)
df2_hierarchy_eta_45_normal.csv (2048 bytes)
df2_hierarchy_eta_exponential5.csv (1869 bytes)
df2_hierarchy_eta_uniform5.csv (2237 bytes)
df2_hierarchy_native_exponential_bias.csv (835 bytes)
df2_hierarchy_native_normal.csv (876 bytes)
df2_hierarchy_native_normal_bias.csv (870 bytes)
df2_hierarchy_native_uniform.csv (866 bytes)
df2_hierarchy_native_uniform_bias.csv (860 bytes)


## Analisi Struttura delle Gerarchie

In [3]:
# Analizza la struttura di ogni file di gerarchia
print("ANALISI STRUTTURA GERARCHIE")
print("=" * 50)

hierarchy_info = []

for file in sorted(hierarchy_files):
    print(f"\n{file.name}:")

    try:
        # Prova prima con separatore semicolon
        df = pd.read_csv(file, header=None, sep=';', dtype=str)
        separator_used = ';'
    except:
        try:
            # Se fallisce, prova con virgola
            df = pd.read_csv(file, header=None, sep=',', dtype=str)
            separator_used = ','
        except Exception as e:
            print(f" Errore nel caricamento: {e}")
            continue

    # Riempi valori mancanti
    df_filled = df.fillna('*')

    print(f" Dimensioni: {df.shape}")
    print(f" Separatore: '{separator_used}'")
    print(f" Livelli gerarchia: {df.shape[1]}")
    print(f" Valori unici nel livello base: {df[0].nunique()}")

    # Mostra prime righe
    print(f" Prime 3 righe:")
    for i in range(min(3, len(df))):
        row_data = df_filled.iloc[i].tolist()
        print(f"    {i}: {row_data}")

    # Salva info per riepilogo
    name_parts = file.stem.split('_hierarchy_')
    if len(name_parts) == 2:
        dataset, column = name_parts
        hierarchy_info.append({
            'File': file.name,
            'Dataset': dataset,
            'Colonna': column,
            'Righe': df.shape[0],
            'Livelli': df.shape[1],
            'Separatore': separator_used,
            'Status': 'OK'
        })
    else:
        hierarchy_info.append({
            'File': file.name,
            'Dataset': 'Parse Error',
            'Colonna': 'Parse Error',
            'Righe': df.shape[0],
            'Livelli': df.shape[1],
            'Separatore': separator_used,
            'Status': 'Nome non standard'
        })

# Crea tabella riassuntiva
if hierarchy_info:
    hierarchy_df = pd.DataFrame(hierarchy_info)
    print(f"\n TABELLA RIASSUNTIVA GERARCHIE:")
    display(hierarchy_df)

ANALISI STRUTTURA GERARCHIE

df1_hierarchy_eta_45_normal.csv:
 Dimensioni: (96, 5)
 Separatore: ';'
 Livelli gerarchia: 5
 Valori unici nel livello base: 96
 Prime 3 righe:
    0: ['63', '60-64', '60-69', '60-79', '*']
    1: ['57', '55-59', '50-59', '40-59', '*']
    2: ['34', '30-34', '30-39', '20-39', '*']

df1_hierarchy_eta_70_normal.csv:
 Dimensioni: (36, 5)
 Separatore: ';'
 Livelli gerarchia: 5
 Valori unici nel livello base: 36
 Prime 3 righe:
    0: ['70', '70-74', '70-79', '60-79', '*']
    1: ['57', '55-59', '50-59', '40-59', '*']
    2: ['65', '65-69', '60-69', '60-79', '*']

df1_hierarchy_eta_exponential5.csv:
 Dimensioni: (86, 5)
 Separatore: ';'
 Livelli gerarchia: 5
 Valori unici nel livello base: 86
 Prime 3 righe:
    0: ['54', '50-54', '50-59', '40-59', '*']
    1: ['29', '25-29', '20-29', '20-39', '*']
    2: ['45', '45-49', '40-49', '40-59', '*']

df1_hierarchy_eta_uniform5.csv:
 Dimensioni: (100, 5)
 Separatore: ';'
 Livelli gerarchia: 5
 Valori unici nel livello 

,File,Dataset,Colonna,Righe,Livelli,Separatore,Status
0,df1_hierarchy_eta_45_normal.csv,df1,eta_45_normal,96,5,;,OK
1,df1_hierarchy_eta_70_normal.csv,df1,eta_70_normal,36,5,;,OK
2,df1_hierarchy_eta_exponential5.csv,df1,eta_exponential5,86,5,;,OK
3,df1_hierarchy_eta_uniform5.csv,df1,eta_uniform5,100,5,;,OK
4,df1_hierarchy_native_exponential.csv,df1,native_exponential,38,3,;,OK
5,df1_hierarchy_native_exponential_bias.csv,df1,native_exponential_bias,38,3,;,OK
6,df1_hierarchy_native_normal.csv,df1,native_normal,42,3,;,OK
7,df1_hierarchy_native_uniform.csv,df1,native_uniform,41,3,;,OK
8,df1_hierarchy_native_uniform_bias.csv,df1,native_uniform_bias,41,3,;,OK
9,df2_hierarchy_eta_45_normal.csv,df2,eta_45_normal,91,5,;,OK


## Mappatura Dataset-Colonne

In [4]:
# Crea mappatura delle gerarchie per dataset e colonne
print(" MAPPATURA GERARCHIE PER DATASET")
print("=" * 40)

# Assicurati che hierarchy_files sia una lista di Path
# es: hierarchy_files = sorted(base_dir.glob("**/*_hierarchy_*.csv"))
if 'hierarchy_files' not in globals() or not hierarchy_files:
    print(" 'hierarchy_files' non definito o vuoto.")
    dataset_mapping = {}
else:
    dataset_mapping = defaultdict(set)  # usa set per evitare duplicati

    for file in hierarchy_files:
        # split solo la prima occorrenza
        name_parts = file.stem.split('_hierarchy_', 1)
        if len(name_parts) == 2:
            dataset = name_parts[0]
            column = name_parts[1]
            dataset_mapping[dataset].add(column)
        else:
            print(f" Nome non conforme: {file.name}")

    # converti i set in liste ordinate
    dataset_mapping = {ds: sorted(list(cols)) for ds, cols in dataset_mapping.items()}

# Stampa mappatura
for dataset, columns in sorted(dataset_mapping.items()):
    print(f"\n {dataset}:")
    for col in columns:
        print(f"  - {col}")

print(f"\n Statistiche:")
print(f"  - Dataset con gerarchie: {len(dataset_mapping)}")
total_hierarchies = sum(len(cols) for cols in dataset_mapping.values())
print(f"  - Totale gerarchie: {total_hierarchies}")

 MAPPATURA GERARCHIE PER DATASET

 df1:
  - eta_45_normal
  - eta_70_normal
  - eta_exponential5
  - eta_uniform5
  - native_exponential
  - native_exponential_bias
  - native_normal
  - native_uniform
  - native_uniform_bias

 df2:
  - eta_45_normal
  - eta_exponential5
  - eta_uniform5
  - native_exponential_bias
  - native_normal
  - native_normal_bias
  - native_uniform
  - native_uniform_bias

 Statistiche:
  - Dataset con gerarchie: 2
  - Totale gerarchie: 17


## Test di Conversione per ARX

In [5]:
# Test della conversione in formato compatibile ARX
print("TEST CONVERSIONE FORMATO ARX")
print("=" * 35)

def test_arx_conversion(filename):
    """
    Testa se un file può essere convertito nel formato richiesto da ARX
    """
    print(f"\nTest: {filename}")

    try:
        # Carica file
        df = pd.read_csv(filename, header=None, sep=';', dtype=str)
        df = df.fillna('*')

        # Converti in lista (formato ARX)
        rows = df.values.tolist()

        print(f" Caricamento riuscito")
        print(f" Dimensioni: {df.shape}")
        print(f" Conversione lista: OK")
        print(f" Esempio riga: {rows[0] if rows else 'Nessuna riga'}")

        # Verifica struttura gerarchia
        if len(rows) > 0 and len(rows[0]) >= 2:
            print(f" Struttura gerarchia: VALIDA")
            print(f" Livelli: {len(rows[0])}")
        else:
            print(f" Struttura gerarchia: QUESTIONABILE")

        return True, df.shape, len(rows[0]) if rows else 0

    except Exception as e:
        print(f" Errore: {e}")
        return False, None, 0

# Testa alcuni file come esempio
test_files = [
    'df2_hierarchy_native_normal.csv',
    'df2_hierarchy_eta_45_normal.csv',
    'df2_hierarchy_eta_exponential5.csv'
]

conversion_results = []
path = base_dir / "df2"
for filename in test_files:
    if (path / filename).exists():
        success, shape, levels = test_arx_conversion(path / filename)
        conversion_results.append({
            'File': filename,
            'Conversione': 'OK' if success else 'ERRORE',
            'Righe': shape[0] if shape else 0,
            'Livelli': levels,
            'Pronto per ARX': 'SÌ' if success and levels >= 2 else 'NO'
        })
    else:
        print(f"\n {filename}: File non trovato")
        conversion_results.append({
            'File': filename,
            'Conversione': 'FILE NON TROVATO',
            'Righe': 0,
            'Livelli': 0,
            'Pronto per ARX': 'NO'
        })

# Tabella risultati conversione
if conversion_results:
    conversion_df = pd.DataFrame(conversion_results)
    print(f"\n RISULTATI TEST CONVERSIONE:")
    display(conversion_df)

# Conta file pronti per ARX
ready_files = sum(1 for r in conversion_results if r['Pronto per ARX'] == 'SÌ')
print(f"\n File pronti per ARX: {ready_files}/{len(conversion_results)}")

TEST CONVERSIONE FORMATO ARX

Test: /content/drive/MyDrive/Schema Matching/hierarchies/df2/df2_hierarchy_native_normal.csv
 Caricamento riuscito
 Dimensioni: (42, 3)
 Conversione lista: OK
 Esempio riga: ['USA', 'North America', '*']
 Struttura gerarchia: VALIDA
 Livelli: 3

Test: /content/drive/MyDrive/Schema Matching/hierarchies/df2/df2_hierarchy_eta_45_normal.csv
 Caricamento riuscito
 Dimensioni: (91, 5)
 Conversione lista: OK
 Esempio riga: ['57', '55-59', '50-59', '40-59', '*']
 Struttura gerarchia: VALIDA
 Livelli: 5

Test: /content/drive/MyDrive/Schema Matching/hierarchies/df2/df2_hierarchy_eta_exponential5.csv
 Caricamento riuscito
 Dimensioni: (84, 5)
 Conversione lista: OK
 Esempio riga: ['6', '5-9', '0-9', '0-19', '*']
 Struttura gerarchia: VALIDA
 Livelli: 5

 RISULTATI TEST CONVERSIONE:


,File,Conversione,Righe,Livelli,Pronto per ARX
0,df2_hierarchy_native_normal.csv,OK,42,3,SÌ
1,df2_hierarchy_eta_45_normal.csv,OK,91,5,SÌ
2,df2_hierarchy_eta_exponential5.csv,OK,84,5,SÌ



 File pronti per ARX: 3/3


## Verifica e validazione file gerarchie per anonimizzazione

In [6]:
if not hierarchy_files:
    print(" Nessun file di gerarchia trovato!")
else:
    print(" File gerarchie trovati")
    print(f" Totale: {len(hierarchy_files)} file")

    valid_files = sum(1 for r in conversion_results if r.get('Pronto per ARX') == 'SÌ')
    print(f" Validi per ARX: {valid_files}")

    if valid_files > 0:
        print("\n File della gerarchia hanno formato corretto")
    else:
        print(" Alcuni file delle gerarchie hanno problemi di formato")


 File gerarchie trovati
 Totale: 17 file
 Validi per ARX: 3

 File della gerarchia hanno formato corretto
